### Notebook for Segmenring ad clustering Neighborhood in Toronto

Import necessary packages 

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


### Part 1: Scraping the content from Wiki page 

In [34]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned":
        # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

# Dataframe with 3 columns
df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
df.head(12)


,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
5,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
6,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"
7,M8A\n,Not assigned\n,Not assigned\n
8,M9A\n,Etobicoke\n,"Islington Avenue, Humber Valley Village\n"
9,M1B\n,Scarborough\n,"Malvern, Rouge\n"


In [35]:
df[df.Borough!= 'Not assigned\n'].head(12)

,PostalCode,Borough,Neighborhood
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
5,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
6,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"
8,M9A\n,Etobicoke\n,"Islington Avenue, Humber Valley Village\n"
9,M1B\n,Scarborough\n,"Malvern, Rouge\n"
11,M3B\n,North York\n,Don Mills\n
12,M4B\n,East York\n,"Parkview Hill, Woodbine Gardens\n"
13,M5B\n,Downtown Toronto\n,"Garden District, Ryerson\n"


Group all neighborhoods with the same postal code

In [36]:
df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()


df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df["PostalCode"] = df["PostalCode"].str.replace("\n","")

df[df.Borough!= 'Not assigned\n'].head()



,PostalCode,Borough,Neighborhood
1,M1B,Scarborough\n,"Malvern, Rouge"
2,M1C,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough\n,Woburn
5,M1H,Scarborough\n,Cedarbrae


In [37]:
print("Shape: ", df.shape)

Shape:  (180, 3)


### Part 2:  Getting the lattitude and longitudes of each neighborhood

In [48]:
df_geo_coor=pd.read_csv(r'C:\Users\Prerana Karanth\Downloads\Geospatial_Coordinates.csv')
df_geo_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [53]:
df_toronto = pd.merge(df, df_geo_coor, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
# remove the "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)


df_toronto[df_toronto.Borough!= 'Not assigned\n'].head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
1,M1B,Scarborough\n,"Malvern, Rouge",43.806686,-79.194353
2,M1C,Scarborough\n,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
3,M1E,Scarborough\n,"Guildwood, Morningside, West Hill",43.763573,-79.188711
4,M1G,Scarborough\n,Woburn,43.770992,-79.216917
5,M1H,Scarborough\n,Cedarbrae,43.773136,-79.239476
6,M1J,Scarborough\n,Scarborough Village,43.744734,-79.239476
7,M1K,Scarborough\n,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
8,M1L,Scarborough\n,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
9,M1M,Scarborough\n,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
10,M1N,Scarborough\n,"Birch Cliff, Cliffside West",43.692657,-79.264848


### Part 3: Explore and cluster neighborhoods in Toronto

In [54]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


##### Create a map of the whole Toronto City with neighborhoods.

In [55]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

#### Map of part of Toronto city

In [60]:
df_toronto_denc = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_denc.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto\n,The Beaches,43.676357,-79.293031
1,M4K,East Toronto\n,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto\n,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto\n,Studio District,43.659526,-79.340923
4,M4N,Central Toronto\n,Lawrence Park,43.728020,-79.388790


In [61]:
map_toronto_denc = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        df_toronto_denc['Latitude'], 
        df_toronto_denc['Longitude'], 
        df_toronto_denc['Borough'], 
        df_toronto_denc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_denc)  

map_toronto_denc